In [ ]:
from functions import process_motus, MAP_THIGH_TRUNK
from functions import get_confusion_matrix, report
import pandas as pd

_, thigh_trunk_adults = process_motus(
    'thigh+trunk_adults',
    vendor='Other',
    map=MAP_THIGH_TRUNK,
    trunk_sensor=True,
    rotate=False,
    orientation=True,
)

labels = ['lie', 'sit', 'stand', 'shuffle', 'walk', 'stairs', 'run', 'bicycle']


thigh_trunk_adults = pd.read_parquet('thigh+trunk_adults/processed_thigh+trunk.parquet')
thigh_trunk_adults['dataset'] = 'thigh_trunk_adults'
thigh_trunk_adults_cm = get_confusion_matrix(
    true=thigh_trunk_adults['ground_truth'],
    pred=thigh_trunk_adults['activity'],
    labels=labels,
    title='Thigh/Back (Adults, Free-living)',
    color='greens',
    hide_yaxis=False,
    hide_xaxis_title=False,
)

thigh_trunk_adults_performance = report(
    thigh_trunk_adults, true='ground_truth', pred='activity', labels=labels, subject_id='id'
)
thigh_trunk_adults_performance['dataset'] = 'thigh_trunk_adults'

thigh_trunk_adults_cm

In [ ]:
import optuna
from functions import process_motus, MAP_THIGH_TRUNK

# Your custom function signature example:
# def custom_classifier(param1, param2, ...):
#     # logic here
#     return predicted_labels


def objective(trial):
    _, thigh_trunk_adults = process_motus(
        'thigh+trunk_adults',
        vendor='Other',
        map=MAP_THIGH_TRUNK,
        trunk_sensor=True,
        rotate=False,
        orientation=True,
    )

    labels = ['lie', 'sit', 'stand', 'shuffle', 'walk', 'stairs', 'run', 'bicycle']

    thigh_trunk_adults = pd.read_parquet('thigh+trunk_adults/processed_thigh+trunk.parquet')
    thigh_trunk_adults['dataset'] = 'thigh_trunk_adults'
    thigh_trunk_adults_cm = get_confusion_matrix(
        true=thigh_trunk_adults['ground_truth'],
        pred=thigh_trunk_adults['activity'],
        labels=labels,
        title='Thigh/Back (Adults, Free-living)',
        color='greens',
        hide_yaxis=False,
        hide_xaxis_title=False,
    )

    thigh_trunk_adults_performance = report(
        thigh_trunk_adults, true='ground_truth', pred='activity', labels=labels, subject_id='id'
    )
    thigh_trunk_adults_performance['dataset'] = 'thigh_trunk_adults'

    return thigh_trunk_adults_performance.loc[('mean', 'accuracy'), 'lie']


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print('Best params:', study.best_params)
print('Best precision:', study.best_value)

In [ ]:
import tomllib
from pydantic import BaseModel, field_validator
from datetime import timedelta
from typing import Literal, Self
import pandas as pd
from pathlib import Path


def string_to_path(value: str | Path) -> Path:
    if isinstance(value, Path):
        return value

    return Path(value)


def string_to_timedelta(value: str) -> timedelta:
    return pd.Timedelta(value).to_pytimedelta()


class Chunking(BaseModel):
    enabled: bool
    size: timedelta
    overlap: timedelta

    @field_validator('size', 'overlap', mode='before')
    def validate_timedelta(cls, value: str) -> timedelta:
        return string_to_timedelta(value)


class Core(BaseModel):
    sampling_frequency: float
    validation: bool
    chunking: Chunking


class Features(BaseModel):
    resample: Literal['fft', 'legacy']
    calibrate: bool


class Config(BaseModel):
    core: Core
    features: Features

    @classmethod
    def from_toml(cls, path: str | Path) -> Self:
        path = string_to_path(path)
        with open(path, 'rb') as f:
            config = tomllib.load(f)

        return cls(**config)


config = Config.from_toml('../../config.toml')
config.features